In [1]:
import numpy as np
import keras

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input,Embedding,Dense,Conv2D,MaxPool2D

In [5]:
from tensorflow.keras.applications import VGG16

In [7]:
import numpy as np

base_sizes = [32, 64, 128, 256, 512]
ratios = [0.5, 1, 2]
scales = [2 ** 0, 2 ** (1/3), 2 ** (2/3)]  

anchor_boxes = []

for base_size in base_sizes:
    for ratio in ratios:
        for scale in scales:
            width = base_size * scale * np.sqrt(ratio)
            height = base_size * scale / np.sqrt(ratio)
            anchor_boxes.append([width, height])

anchor_boxes = np.array(anchor_boxes)

print("Number of anchor boxes:", len(anchor_boxes))
#print("Anchor box shapes:\n", anchor_boxes)


Number of anchor boxes: 45


In [8]:
input_image = Input(shape=(None, None, 3))
backbone = VGG16(include_top=False, input_tensor=input_image)
feature_map = backbone.output

58889256/58889256 [==============================] - 27s 0us/step


In [9]:
num_anchors=len(anchor_boxes)

In [10]:
rpn_conv = Conv2D(filters=num_anchors * 2, kernel_size=(1, 1), activation='sigmoid')(feature_map)
rpn_bbox = Conv2D(filters=num_anchors * 4, kernel_size=(1, 1))(feature_map)

In [ ]:
def apply_nms_on_rpn(rpn_bbox_offsets, rpn_objectness, num_pre_nms_proposals, num_post_nms_proposals, iou_threshold):
    selected_indices = tf.image.non_max_suppression(
        boxes=rpn_bbox_offsets,
        scores=rpn_objectness[:, 1],
        max_output_size=num_pre_nms_proposals,
        iou_threshold=iou_threshold
    )
    
    selected_indices = selected_indices[:num_post_nms_proposals]
    selected_proposals = tf.gather(rpn_bbox_offsets, selected_indices)
    
    return selected_proposals

num_pre_nms_proposals = 2000
num_post_nms_proposals = 1000
iou_threshold = 0.7

selected_proposals = apply_nms_on_rpn(rpn_bbox_offsets, rpn_objectness, num_pre_nms_proposals, num_post_nms_proposals, iou_threshold)
print("Selected proposals after NMS:", selected_proposals)
